In [321]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [322]:
import numpy as np
from timer import Timer
import pickle

In [323]:
maxFloat16 = np.finfo(np.float16).max
neuronMax = 10
neuronMin = -10
neuronDtype = np.float32


class NeuralNetwork:
    @staticmethod
    def sigmoid(x, derivative=False):
        if derivative:
            return x * (1 - x)
        return 1 / (1 + np.exp(-x))

    @staticmethod
    def relu(x, derivative=False):
        if derivative:
            return 1.0 #* (x > 0)
        return x #np.maximum(0, x)


    # @staticmethod
    # def softmax(x):
    #     exps = np.exp(x - np.max(x))
    #     return exps / np.sum(exps, axis=1, keepdims=True)

    # @staticmethod
    # def softmaxDerivative(x):
    #     return x * (1 - x)

    def __init__(self, layerSizes, activationFunctions):
        self.layerSizes = layerSizes
        self.activationFunctions = activationFunctions
        self.layers = [
            np.random.uniform(low=neuronMin, high=neuronMax, size=layerSizes[i]).astype(neuronDtype)
            for i in range(len(layerSizes))
        ]
        self.weights = [
            np.random.uniform(
                low=neuronMin, high=neuronMax, size=layerSizes[i] * layerSizes[i + 1]
            ).astype(neuronDtype).reshape(layerSizes[i], layerSizes[i + 1])
            for i in range(len(layerSizes) - 1)
        ]
        self.biases = [
            np.random.uniform(low=neuronMin, high=neuronMax, size=layerSizes[i + 1]).astype(
                neuronDtype
            )
            for i in range(len(layerSizes) - 1)
        ]

    def loadInput(self, input):
        self.layers[0] = input.astype(neuronDtype)

    def readOutput(self):
        return self.layers[-1]

    def forward(self):
        for i in range(len(self.weights)):
            self.layers[i + 1] = self.activationFunctions[i](
                np.dot(self.layers[i], self.weights[i]) + self.biases[i]
            )

    def mseLoss(self, target, layer_no):
        return np.sum(np.square(target - self.layers[layer_no]))/2

    def backpropagation(self, target, learning_rate):
        output_error = target - self.layers[-1]

        for i in range(len(self.layers) - 2, -1, -1):
            # Calculate the derivative of the activation function
            derivative = self.activationFunctions[i](self.layers[i + 1], derivative=True)

            # Calculate the error of the current layer
            layer_error = np.dot(output_error * derivative, self.weights[i].T)

            # Calculate the adjustments for the weights and biases
            weight_adjustments = np.outer(self.layers[i], output_error)
            bias_adjustments = output_error

            # Update the weights and biases
            self.weights[i] += weight_adjustments * learning_rate
            self.biases[i] += bias_adjustments * learning_rate

            # Set the output error for the next iteration
            output_error = layer_error * learning_rate#derivative


In [324]:
# x = NeuralNetwork([6, 5, 3], [NeuralNetwork.relu, NeuralNetwork.relu])
# nnTimer = Timer(precision=3)
# target = np.array([2, 3, 4])
# input = np.random.rand(6)


# x = NeuralNetwork([4_107, 40, 40, 3], [NeuralNetwork.relu, NeuralNetwork.relu, NeuralNetwork.relu])
# nnTimer = Timer(precision=3)
# target = np.array([2, 3, 4])
# input = np.random.rand(4_107)

x = NeuralNetwork([300_000, 40, 40, 3], [NeuralNetwork.relu, NeuralNetwork.relu, NeuralNetwork.relu])
nnTimer = Timer(precision=3)
target = np.array([2, 3, 4])
input = np.random.rand(300_000)


# x = NeuralNetwork([12, 10, 10, 25], [NeuralNetwork.relu, NeuralNetwork.relu, NeuralNetwork.relu])
# nnTimer = Timer(precision=3)
# target = np.arange(25)
# input = np.random.rand(12)



# for i in range(1):
#     nnTimer.tic()
#     x.loadInput(np.random.rand(30_000))
#     x.forward()
#     nnTimer.toc()
#     print(x.readOutput())
# nnTimer.stats()

In [325]:
x.loadInput(input)
x.forward()
print(x.readOutput())

[-3979031.8   -903105.25  2305689.2 ]


In [365]:
import pandas as pd

results = pd.DataFrame(
    columns=[
        "trialID",
        "ticTime",
        "totalTime",
        "learningRates",
        "inputSize",
        "target",
        "outputs",
    ]
)

learning_rates = [0.1, 0.01, 0.001, 0.0001, 0.0000000000001, 0.0000000000001]
inputSizes = [12, 500, 5_000, 50_000, 500_000]


testCase = 0
for learning_rate in learning_rates:
    for inputSize in inputSizes:
        x = NeuralNetwork(
            [inputSize, 40, 40, 3],
            [NeuralNetwork.relu, NeuralNetwork.relu, NeuralNetwork.relu],
        )
        nnTimer = Timer(precision=3)
        target = np.arange(3)
        input = np.random.rand(inputSize)
        outputs = []

        i = 0
        while (
            i < 10_000
            and nnTimer.getElapsedTime() < 100_000
            and (x.readOutput().mean() not in (np.inf, -np.inf, np.nan))
            # and sum([ (res < targ * 1.1 and res > targ * 1.1 ) for res, targ in zip(x.readOutput(), target)])
        ):
            nnTimer.tic()
            x.loadInput(input)
            x.forward()
            x.backpropagation(target, learning_rate)
            nnTimer.toc()
            outputs.append(x.readOutput())
            i += 1

        results.loc[testCase] = [
            testCase,
            nnTimer.tic_time,
            nnTimer.total_time,
            learning_rate,
            inputSize,
            target,
            outputs,
        ]

        # (
        #     {
        #         "trialID": testCase,
        #         "ticTime": nnTimer.tic_time,
        #         "totalTime": nnTimer.total_time,
        #         "learningRates": learning_rate,
        #         "inputSize": inputSize,
        #         "target": target,
        #         "outputs": outputs,
        #     },
        # )

        if testCase % 10 == 0:
            with open(f"profiling/results{testCase/10}.pkl", "wb") as f:
                pickle.dump(results, f)

        testCase += 1

C:\Users\milos\AppData\Local\Temp\ipykernel_20112\3886743066.py:80: RuntimeWarning: overflow encountered in add
  self.weights[i] += weight_adjustments * learning_rate
C:\Users\milos\AppData\Local\Temp\ipykernel_20112\3886743066.py:81: RuntimeWarning: overflow encountered in add
  self.biases[i] += bias_adjustments * learning_rate
C:\Users\milos\AppData\Local\Temp\ipykernel_20112\3886743066.py:80: RuntimeWarning: overflow encountered in add
  self.weights[i] += weight_adjustments * learning_rate
C:\Users\milos\AppData\Local\Temp\ipykernel_20112\3886743066.py:81: RuntimeWarning: overflow encountered in add
  self.biases[i] += bias_adjustments * learning_rate
C:\Users\milos\AppData\Local\Temp\ipykernel_20112\3886743066.py:80: RuntimeWarning: overflow encountered in add
  self.weights[i] += weight_adjustments * learning_rate
C:\Users\milos\AppData\Local\Temp\ipykernel_20112\3886743066.py:81: RuntimeWarning: overflow encountered in add
  self.biases[i] += bias_adjustments * learning_rate
C

In [327]:
nnTimer.stats()

Total time: 0.705 s
Total calls: 20
Average time: 0.03525 s


In [ ]:
learning_rate = 0.0000000000001
for i in range(20):
    x.backpropagation(target, learning_rate)
    nnTimer.tic()
    x.loadInput(input)
    x.forward()
    print(x.readOutput())
    nnTimer.toc()

In [328]:
import time

In [329]:
time.process_time()

215.703125

In [339]:
time.process_time()

216.671875

In [357]:
timer = Timer(precision=3)
timer.tic()
x = 0
for i in range(1000000):
    x+=x+1
timer.toc()
timer.getElapsedTime()

19703

78

In [360]:
results = pd.DataFrame(
    columns=[
        "trialID",
        "ticTime",
        "totalTime",
        "learningRates",
        "inputSize",
        "target",
        "outputs",
    ]
)

In [363]:
results.loc[0] = (
    {
        "trialID": 0,
        "ticTime": 0.1,
        "totalTime": 0.2,
        "learningRates": 0.1,
        "inputSize": 12,
        "target": np.arange(3),
        "outputs": np.random.rand(3),
    },
)

IndexError: iloc cannot enlarge its target object